## 전체 화면에서 흰 화면 뽑아내기 (개선 전 버젼)

In [ ]:
import cv2
import numpy as np


def white_img_extract(FILENAME):
    """전체 화면에서 흰 화면만 뽑아내기

    Args:
        FILENAME (_type_): 파일이름

    Returns:
        _type_: ndarray
    """
    img = cv2.imread(FILENAME)

    # 전처리 과정
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3, 3), 0)
    thresh = cv2.Canny(blur, 50, 200)

    # 윤곽선 찾아내기
    contours, hierarchy = cv2.findContours(
        thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE
    )
    # contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    # 이미지 크롭
    sl = []
    box_sl = []

    for cnt in contours:
        pt, hw, ang = cv2.minAreaRect(cnt)
        # minAreaRect 입력값: contours, 반환값: 좌측상단 점, (가로,세로), 각도
        if hw[0] > 1000 and hw[1] > 500:
            # if hw[0]>500:
            sl.append([pt, hw, ang])

    for i in sl:
        box = cv2.boxPoints((i[0], i[1], i[2]))
        box_sl.append(np.int0(box))

    # 이미지 보여주기
    for box in box_sl:
        img = cv2.drawContours(img, [box], 0, (0, 255, 0), 3)

    cv2.namedWindow("img", flags=cv2.WINDOW_NORMAL)
    cv2.imshow("img", img)
    cv2.waitKey()
    cv2.destroyAllWindows()

    # 원근변환 이용해서 필요한 이미지 추출
    box = np.float32(box)
    next_arr = np.array([[0, 0], [1200, 0], [1200, 1600], [0, 1600]], dtype=np.float32)
    per_mat = cv2.getPerspectiveTransform(box, next_arr)
    per = cv2.warpPerspective(img, per_mat, (1200, 1600))

    # 원근변환한 이미지 보여주기
    cv2.imshow("per", per)
    cv2.waitKey()
    cv2.destroyAllWindows()
    return per


# 예시
if "__main__" == __name__:
    white_img_extract("case2_2.jpg")

## 전체 화면에서 흰 화면 뽑아내기 (개선 후 버젼)
- 'best.jpg'라는 true_ok에서 임의로 선정한 샘플 사용

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
def make_mask(per, n):
    """ 이미지에 마진margin을 n만큼 설정해서 출력

    Args:
        per (ndarray): 이미지
        n (int): 마진margin 

    Returns:
        ndarray: 마진을 설정한 이미지
    """
    mask = np.zeros(per.shape[:2], np.uint8)
    mask[n : per.shape[0] - n, n : per.shape[1] - n] = 255
    return mask


def white_select(FILENAME, graph=True):
    """matchTemplate 함수를 사용해서 전체 이미지에서 흰 화면 추출하는 함수

    Args:
        FILENAME (str): 파일경로
        graph (bool, optional): 그래프 출력 유무. Defaults to True.

    Returns:
        ndarray: 흰 화면 이미지(검은 부분 조금 있음)
    """
    img = cv2.imread("best.jpg")    # 베스트 파일 필요
    best = img[365:1635,383:2070].copy()

    img2 = cv2.imread(FILENAME)
    best = cv2.cvtColor(best, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    tm_sqdiff = cv2.matchTemplate(img2, best, cv2.TM_SQDIFF)
    min_val_diff, max_val_diff, min_loc_diff, max_loc_diff = cv2.minMaxLoc(tm_sqdiff)
    h, w = best.shape

    
    cv2.rectangle(
        img2,
        (min_loc_diff[0], min_loc_diff[1]),
        (min_loc_diff[0] + w, min_loc_diff[1] + h),
        (0, 255, 0),
    )
    img_crop = img2[
        min_loc_diff[1]: min_loc_diff[1] + h, min_loc_diff[0] : min_loc_diff[0] + w
    ].copy()
    if graph == True:
        cv2.namedWindow("img_crop", flags=cv2.WINDOW_NORMAL)
        cv2.imshow("img_crop", img_crop)
        cv2.waitKey()
        cv2.destroyAllWindows()
    return img_crop
    # cv2.namedWin/dow('img2', flags=cv2.WINDOW_NORMAL)
    # cv2.namedWindow("best", flags=cv2.WINDOW_NORMAL)
    
    # cv2.imshow("best", best)
    # cv2.imshow("img2", img2)
    # cv2.imshow("tm_sqdiff", tm_sqdiff/np.max(tm_sqdiff))
    
def rect_extract(FILENAME, graph=True):
    """minAreaRect 함수를 사용해서 white_extract 함수를 적용한 이미지에서, 흰 센서만 추출

    Args:
        FILENAME (str): 이미지파일경로
        graph (bool, optional): 그래프 출력 유무. Defaults to True.

    Returns:
        ndarray: 센서 이미지
    """
    img = white_select(FILENAME, graph=graph)
    blur = cv2.GaussianBlur(img, (3, 3), 0)
    thresh = cv2.Canny(blur, 50, 200)

    contours, hierarchy = cv2.findContours(
            thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE
        )

    sl = []
    box_sl = []

    for cnt in contours:
        pt, hw, ang = cv2.minAreaRect(cnt)
        # minAreaRect 입력값: contours, 반환값: 좌측상단 점, (가로,세로), 각도
        if hw[0] > 1000 and hw[1] > 500:
            sl.append([pt, hw, ang])

    for i in sl:
        box = cv2.boxPoints((i[0], i[1], i[2]))
        box_sl.append(np.int0(box))
    
    try: 
        for box in box_sl:
            img = cv2.drawContours(img, [box], 0, (0, 255, 0), 3)
            if graph:
                cv2.namedWindow("img", flags=cv2.WINDOW_NORMAL)
                cv2.imshow("img", img)
                cv2.waitKey()
                cv2.destroyAllWindows()

        box = np.float32(box)
        next_arr = np.array([[0, 0], [1200, 0], [1200, 1600], [0, 1600]], dtype=np.float32)
        per_mat = cv2.getPerspectiveTransform(box, next_arr)
        per = cv2.warpPerspective(img, per_mat, (1200, 1600))
    except Exception as e:
        return []
    # 원근변환한 이미지 보여주기
    if graph:
        cv2.namedWindow("per", flags=cv2.WINDOW_NORMAL)
        cv2.imshow("per", per)
        cv2.waitKey()
        cv2.destroyAllWindows()
    return per

def check_model2(imgpath, graph=True, num=160):
    """조건2 체크하는 함수: rect_extract 함수를 통과시킨 센서 이미지의
    색 분포 히스토그램을 calcHist를 이용해서 계산 후 num 기준 이하의 이미지만
    True, 아니면 False로 반환하는 함수

    Args:
        imgpath (str): 이미지 파일 경로
        graph (bool, optional): 그래프 출력 유무. Defaults to True.
        num (int, optional): 히스토그램 색 분포 기준. Defaults to 160.

    Returns:
        _type_: _description_
    """
    per = rect_extract(imgpath, graph=graph)
    # rect_extract 통과한 이미지 출력했을 때 반환되는 이미지가 없는 경우
    if len(per) == 0:
        # return np.inf
        return False
    else:
        # rect_extract 통과한 이미지 출력했을 때 반환되는 이미지가 있는 경우
        n = 20  # 전체 이미지에서 마진margin을 얼만큼 띄울건지 체크
        mask = make_mask(per, n)
        hist = cv2.calcHist([per], [0], mask, [256], [0, 256])
        # return hist[:-6].sum()     # 250~255 정도는 255와 동일시하다고 판단해서 제거
        if hist[:-6].sum() >= num:           # num를 조절
            return False
        else:
            return True


if "__main__" == __name__:
    check_model2("이미지파일주소", graph=True, num=160)
